In [3]:
#0: imports

import uproot 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from imports.data import CCV3
from torch_geometric.data import DataLoader 

from torch_geometric.nn import knn_graph

import numpy as np
import awkward as ak
import time
from imports.Agglomerative import Aggloremative

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [20]:
testpath = '/vols/cms/mm1221/Data/mix/test/'
# Load test data
data_test = CCV3(testpath, max_events=1000)
test_loader = DataLoader(data_test, batch_size=1, shuffle=False, follow_batch=['x'])


### Loading data


  0%|                                                     | 0/1 [01:19<?, ?it/s]

Reached 1000!


In [22]:
v_eta = data_test.stsCP_vertices_eta
v_phi = data_test.stsCP_vertices_phi

In [17]:
data_path = '/vols/cms/mm1221/Data/mix/test/raw/test.root' 
data_file = uproot.open(data_path)
LC_x = data_file['clusters']['position_x'].array()
pgid = data_file['simtrackstersCP']['pdgID'].array()
skim_mask = []
for e in LC_x:
    if 1 <= len(e):
        skim_mask.append(True)
    else:
        skim_mask.append(False)
        
pgid = pgid[skim_mask]

In [18]:
print(pgid[0])

[211, 211]


In [23]:
import numpy as np

# v_eta, v_phi: lists of events → list of particles → array of η (or φ)
# pgid:      lists of events → list of particles → PDG ID (211 or 11)

pion_widths     = []
electron_widths = []

for event_eta, event_phi, event_pgid in zip(v_eta, v_phi, pgid):
    for eta_vals, phi_vals, pid in zip(event_eta, event_phi, event_pgid):
        # compute centroids
        eta0 = np.mean(eta_vals)
        phi0 = np.mean(phi_vals)

        # eta deviations
        d_eta = eta_vals - eta0
        # phi deviations, wrapped into [-π, π)
        d_phi = ((phi_vals - phi0 + np.pi) % (2*np.pi)) - np.pi

        # per-hit ΔR²
        dr2 = d_eta**2 + d_phi**2

        # cluster width = sqrt(mean of ΔR²)
        w = np.sqrt(np.mean(dr2))

        # classify by PDG ID
        if pid == 211:
            pion_widths.append(w)
        elif pid == 11:
            electron_widths.append(w)
        # else: ignore other PIDs if any

# convert to arrays for stats
pion_widths     = np.array(pion_widths)
electron_widths = np.array(electron_widths)

# compute mean and sample std (ddof=1)
mean_pion     = pion_widths.mean()
std_pion      = pion_widths.std(ddof=1)
mean_electron = electron_widths.mean()
std_electron  = electron_widths.std(ddof=1)

print(f"Pions     (N={len(pion_widths)}):     mean = {mean_pion:.4f}, std = {std_pion:.4f}")
print(f"Electrons (N={len(electron_widths)}): mean = {mean_electron:.4f}, std = {std_electron:.4f}")


Pions     (N=3035):     mean = 0.1418, std = 0.2478
Electrons (N=2934): mean = 0.1138, std = 0.2295
